In [503]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## get adj_matrix for later use

In [504]:
def generate_adjacency_matrix(file_name):
    input_graph = np.loadtxt(file_name)
    input_graph = input_graph.astype(int)-1
    size = max(max(input_graph[:,0]),max(input_graph[:,1]))+1
    adjacency_matrix = np.zeros((size,size))
    for pair in input_graph:
        adjacency_matrix[pair[0]][pair[1]] = 1
        adjacency_matrix[pair[1]][pair[0]] = 1
    return adjacency_matrix

## input test file here

In [505]:
adjacency_matrix = generate_adjacency_matrix("professor_test.txt")
size = len(adjacency_matrix)

## solve maximum k-dependent set problem using quadratic programming

In [506]:
def max_k_dependent_set_quadratic(k):
    model = gp.Model("maximum k dependent set")
    x = model.addMVar(shape = size, lb=0, vtype = GRB.BINARY)
    model.setObjective(x.sum(),GRB.MAXIMIZE)

    for i in range(size):
        model.addConstr(x[i]*(gp.quicksum(x[k]*adjacency_matrix[i][k] for k in range(size)))<=k)

    model.Params.OutputFlag = 0
    model.optimize()
    
    return model.ObjVal

## solve maximum k-dependent set problem using linear programming

In [507]:
def max_k_dependent_set_linear(k):
    model = gp.Model("maximum k dependent set")
    x = model.addVars(size, vtype = GRB.BINARY,name="x")
    y = model.addVars(size,size,vtype = GRB.BINARY,name="y")
    model.setObjective(x.sum(), GRB.MAXIMIZE)

    for i in range(size):
        model.addConstrs(y[i,j]<=x[i] for j in range(size))
        model.addConstr((gp.quicksum(y[i,j] for j in range(size) if adjacency_matrix[i][j]))<=k )
    for j in range(size):
        model.addConstrs(y[i,j]<=x[j] for i in range(size))
    for i in range(size):
        for j in range(size):
            model.addConstr(y[i,j]>=x[i]+x[j]-1)

    model.Params.OutputFlag = 0
    model.optimize()
    
    return model.ObjVal

## get results for different k and put them in a list(it's easier to test later)
## you can choose the way you like to solve it using gurobi(quadratic or linear)

In [508]:
results_for_different_k = []
for k in range(1,16): 
    results_for_different_k.append(max_k_dependent_set_linear(k)) 
    #results_for_different_k.append(max_k_dependent_set_quadratic(k)) 
print("Cardinality for different k: ", results_for_different_k)

Cardinality for different k:  [14.0, 14.0, 16.0, 17.0, 18.0, 20.0, 21.0, 23.0, 24.0, 24.0, 25.0, 25.0, 25.0, 25.0, 25.0]


## I assume you put this test code with our assignments in the same file, and you can convert .ipynb file to .py file and directly get assignment results

In [509]:
import Bonus_Xie_H
results_hw = Bonus_Xie_H.main()

def test(results_for_different_k, results_hw):
    for k in range(15):
        if not results_for_different_k[k]/results_hw[k] <= 1+k/(k+2):
            return "Something is wrong with this assignment"
    return "This assignment is correct"

print(test(results_for_different_k, results_hw))

Maximum k-dependent set:
vertex12  vertex13  vertex14  vertex15  vertex16  vertex17  vertex18  vertex19  vertex20  vertex21  vertex22  vertex23  vertex24  vertex25  

Maximum cardinality of k-dependent set in G:  14
----------------------------------------------------------------------------------------------------
Maximum k-dependent set:
vertex12  vertex13  vertex14  vertex15  vertex16  vertex17  vertex18  vertex19  vertex20  vertex21  vertex22  vertex23  vertex24  vertex25  

Maximum cardinality of k-dependent set in G:  14
----------------------------------------------------------------------------------------------------
Maximum k-dependent set:
vertex4  vertex9  vertex10  vertex13  vertex14  vertex15  vertex16  vertex17  vertex18  vertex20  vertex21  vertex22  vertex23  vertex24  

Maximum cardinality of k-dependent set in G:  14
----------------------------------------------------------------------------------------------------
Maximum k-dependent set:
vertex4  vertex8  vertex9 